<img src='./img/logo.png'></img>

# UOPY Example - Working with JSON Best Practices

# Making the connection

The followinf code is a simple example to connect to a local U2 Database.  In this example I am connecting to UniData, but the code would work the same against UniVerse, the only thing you woul dneed to modify is the servce and account settings.

In [4]:
import uopy
import getpass
user = input("Enter user name? ")
passwd = getpass.getpass("Enter Password? ")

def makeconnection(user, password):
    config = {
             'user': user,
             'password': passwd,
             'service': 'udcs',
             'account': 'C:\\U2\\ud83\\XDEMO',
             'encoding': 'GB18030',
             'pooling_on': 'False'
         }
    thissession = uopy.connect(**config)
    return thissession
session = makeconnection(user, passwd)
print(str(session))

<uopy._session.Session object {'host': 'localhost', 'account': 'C:\\U2\\ud83\\XDEMO', 'port': 31438, 'timeout': 300, 'service': 'udcs', 'encoding': 'GB18030', 'ssl': False, 'user': 'mrajkowski', 'password': ''} at 0x0000027A2B9E6610>


# Read in a JSON document

Note that to in this example, I am limiting my discussion on how to create and consume JSON.  I am using the TOJSON keyword to generate the JSON from a U2 File, in this case the MEMBERS file in the XDEMO account.

In [5]:
import pprint

cmd = uopy.Command("LIST REGION NAME REGION_SALES SAMPLE 3 TOJSON")
st = cmd.run()
out = cmd.response
print(out)



{
"REGION":[
 {"_ID" : "3","NAME" : "West","REGION_SALES" : "717038"
 },
 {"_ID" : "1","NAME" : "East","REGION_SALES" : "1202538"
 },
 {"_ID" : "2","NAME" : "Central","REGION_SALES" : "1393565"
 }
]
}



# Converting the string to a Python Dictionary with eval() function

Note that while this is an easy way, it could be exploited by hackers.


In [6]:
data = eval(out)

print(type(data))
pprint.pprint(data)

<class 'dict'>
{'REGION': [{'NAME': 'West', 'REGION_SALES': '717038', '_ID': '3'},
            {'NAME': 'East', 'REGION_SALES': '1202538', '_ID': '1'},
            {'NAME': 'Central', 'REGION_SALES': '1393565', '_ID': '2'}]}


As you can see with one command, I changed the string to a python dictionary object.  The problem arises when you are not in control of what is being passed into the eval function.


In [7]:
code_string = 'print("This is an example of the eval() function executing commands.")'
eval(code_string)

This is an example of the eval() function executing commands.


# Using the Python json module

A better way of dealing with the json string is to marshel it to a Python dictionary with the built-in json module.


In [8]:
import json

try: 
    jdata = json.loads(out)
    print(type(jdata))
    pprint.pprint(jdata)
except:
    print("problem with json")

<class 'dict'>
{'REGION': [{'NAME': 'West', 'REGION_SALES': '717038', '_ID': '3'},
            {'NAME': 'East', 'REGION_SALES': '1202538', '_ID': '1'},
            {'NAME': 'Central', 'REGION_SALES': '1393565', '_ID': '2'}]}


Note that by using the json.loads method, we prevent a non-json string from causing a problem.

In [10]:
try: 
    print("code_string" + code_string)
    jdata = json.loads(code_string)
    print(type(jdata))
    pprint.pprint(jdata)
except:
    print("problem with json")

code_stringprint("This is an example of the eval() function executing commands.")
problem with json


# Once you have the json, you can display some of the information

In [11]:
mydict = dict()
for each in jdata["REGION"]:
    print(each["_ID"] + " is for " + " " + each["NAME"] + ", with sales of " + each["REGION_SALES"])
    mydict[each["_ID"]] = each
    


3 is for  West, with sales of 717038
1 is for  East, with sales of 1202538
2 is for  Central, with sales of 1393565


## Lets look at one of the Python Dict objects a bit closer

In [14]:
this_obj = mydict["1"]
pprint.pprint(this_obj)
print()
print("With the Python dict object, we can access each named-value pair, for example just the NAME:")

print(this_obj["NAME"])

{'NAME': 'East', 'REGION_SALES': '1202538', '_ID': '1'}

With the Python dict object, we can access each named-value pair, for example just the NAME:
East


## Preparing for writing to a new U2 file

Step 1: Get the Keys and the Values
Step 2: Create the file

In [15]:
keys = mydict.keys()
values = mydict.values()
print(keys)
print(values)

dict_keys(['3', '1', '2'])
dict_values([{'_ID': '3', 'NAME': 'West', 'REGION_SALES': '717038'}, {'_ID': '1', 'NAME': 'East', 'REGION_SALES': '1202538'}, {'_ID': '2', 'NAME': 'Central', 'REGION_SALES': '1393565'}])


## Create the U2 file and dictionary ( Note this is a UniData example )


In [17]:
file_needed = 0
filename = "NEW_REGION"
try:
    file = uopy.File(filename)
    dfile = uopy.File(filename, uopy.DICT_FILE )
    print("file opened")
except: 
    print("The file " + filename + " does not exist")
    file_needed = 1

try:
    if file_needed:
        cmd = uopy.Command("CREATE.FILE " + filename + " 3 11")
        cmd.run()
        print("file created")
        file = uopy.File("DATASOURCE")
        dfile = uopy.File(filename, uopy.DICT_FILE )
        print("now the file is opened")
except:
    print("still an issue with the file")
    
print(type(file))

file opened
<class 'uopy._file.File'>


# Create U2 Dictionary items

Note this will step through the keys extracted in the last section and create d-type dictionary items.


In [18]:
one_obj = list(values)[0]
print(type(one_obj))
one_set_of_keys = one_obj.keys()
cnt = 0
for each_key in one_set_of_keys:
    cnt += 1
    print(each_key + " will be location " + str(cnt))
    rec = [ "D", cnt, "", each_key, "10L", "S"]
    dfile.write(each_key, rec)
    
cmd = uopy.Command("LIST DICT " + filename)
cmd.run()
print(cmd.response)

<class 'dict'>
_ID will be location 1
NAME will be location 2
REGION_SALES will be location 3
LIST DICT NEW_REGION BY TYP BY @ID TYP LOC CONV NAME FORMAT SM ASSOC 14:06:17 May 09 2025 1
@ID............ TYP LOC.......... CONV NAME........... FORMAT SM ASSOC.....
 
@ID             D               0      NEW_REGION      10L    S
NAME            D               2      NAME            10L    S
REGION_SALES    D               3      REGION_SALES    10L    S
_ID             D               1      _ID             10L    S
4 records listed
 
















# Now load the data

For this example, I am only loading the first object to the file

In [20]:
this_key = list(keys)[0]
the_fields = list(one_obj.keys())
the_values = [ list(one_obj.values()) ]
print(the_fields) 
print(the_values)
st = file.write_named_fields(this_key, the_fields, the_values)

# Note 'ALL' is only in UniData
cmd = uopy.Command("LIST " + filename + " ALL")
cmd.run()
print(cmd.response)


['_ID', 'NAME', 'REGION_SALES']
[['3', 'West', '717038']]
LIST NEW_REGION ALL 14:07:05 May 09 2025 1
NEW_REGION _ID....... NAME...... REGION_SALES
 
3          3          West       717038
1 record listed
 


















